# Procesamiento de Lenguage Natural

## Taller 8: Modelado de Temas
###    Modelado utilizando la asignación latente de Dirichlet (LDA)

https://es.wikipedia.org/wiki/Peter_Gustav_Lejeune_Dirichlet

https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation

### 1. Se importan las librerias de trabajo warnings para evitar los mensajes de descripción de algunos provcesos

In [10]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### 2. Se realiza el cargue de la infomación, para este ejercicio es un archivo csv.

In [11]:
ruta = "C:/Archivo/"

In [12]:
# Cargar datos
import pandas as pd 

path = ruta  + 'reviews_vidjew_es.csv'
data = pd.read_csv(path)
print(f"El conjunto de datos contiene {len(data)} registros")
data.head()

El conjunto de datos contiene 1000 registros


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


### 3. Se realiza el pre-procesamiento del texto (Feature Engineering)

In [13]:
#Esta libreria permite conocer el porcentaje del o los idiomas en los que se encuentra la información:
from langdetect import detect, detect_langs
vocabulario = data['review_body'].str.cat(sep=" ")
print(detect_langs(vocabulario))    

[es:0.9999973276207876]


In [14]:
#Se carga la libreria para la extracción de palabras vacias, en este ejercicio se evidencia euq el idioma se encuentra en español
import re
from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

In [15]:
#Función generada para realizar el preprocesamiento

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto
data['Pre-Processed'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Processed
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


### 4. Modelo LDA

In [16]:
#importar librerias

import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from pprint import pprint

In [17]:
#Se crea una representación de los documentos en forma de diccionario
dictionary = Dictionary(data['Pre-Processed'].values)

In [18]:
#Se filtran las palabras muy frecuentes y poco frecuentes
#no_above=0.5 -->> la palabra se encuentra en el 50% de los documentos
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in data['Pre-Processed'].values]

In [37]:
#Entrenamiento del modelo
#passes: cantidad de iteraciones, random_state=semilla, num_topics= número de temas
model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=4, passes=4, random_state=42)
#se realizaron varias pruebas en el número de temas en forma descendete, arrancando desde 7,
#sin embargo, se observo sobreposición de temas, por lo cual al final la mejor elección es tomar 4 temas.  
model.print_topics(num_words=10)

[(0,
  '0.026*"bien" + 0.026*"si" + 0.016*"producto" + 0.014*"precio" + 0.013*"mal" + 0.012*"bastante" + 0.011*"solo" + 0.010*"tamaño" + 0.010*"calidad" + 0.010*"parte"'),
 (1,
  '0.039*"bien" + 0.036*"juego" + 0.027*"calidad" + 0.015*"precio" + 0.013*"si" + 0.013*"buena" + 0.012*"caja" + 0.011*"producto" + 0.011*"funciona" + 0.011*"solo"'),
 (2,
  '0.023*"calidad" + 0.023*"precio" + 0.020*"buen" + 0.020*"bonito" + 0.019*"llegó" + 0.013*"bien" + 0.012*"parece" + 0.012*"si" + 0.011*"plata" + 0.011*"producto"'),
 (3,
  '0.023*"llegado" + 0.021*"regalo" + 0.019*"producto" + 0.016*"foto" + 0.015*"bien" + 0.014*"mas" + 0.014*"amazon" + 0.012*"pedido" + 0.012*"calidad" + 0.012*"juego"')]

### 5. Visualización del Modelo

In [38]:
#Tamaño de los circulos esta ligado a la importancia del tema
#Un circulo dentro de otro circulo puede ser porque un tema se encuenta dentro de otro tema
#sort_topics= para que los temas queden ordenados visualmente de acuerdo con el texto contenido en cada tema 
lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=True) 
pyLDAvis.display(lda_display)

In [39]:
#Analisis de probabilidades por palabras en cada uno de los temas
d = dictionary.doc2bow(["bien", "jugar", "calidad", "precio"])
topics = model.get_document_topics(d)
topics
#La probabilidad más alta esta en el segundo tema, lo que quiere decir que estas palabras ejemplo corresponden en mayor probabilidad al segundo tema.

[(0, 0.05246859), (1, 0.84287655), (2, 0.052881755), (3, 0.051773064)]

In [40]:
#Se crea un blucle matriz en donde cada topic(tema) que en la columna
def get_doc_top_n(text_processed, n):
    d = dictionary.doc2bow(text_processed)
    topics = dict(model.get_document_topics(d))
    try:
        return topic[n]
    except:
        return None

In [41]:
#Se crea un blucle matriz en donde cada topic(tema) que en la columna
for t in range(0,3):
    top_name = f"topic_{t}"
    data[top_name] = data['Pre-Processed'].apply(lambda doc: get_doc_top_n(doc,t))

In [42]:
#bucle e imprime los articulos más representativos de cada tema
for t in range(0,4):
    print(f"****************************** TOPIC {t} ******************************")
    for i,row in enumerate(data.sort_values(f"topic_{t}", ascending=False)['review_body'].head()):
        print(f"review_body #{i+1}")
        print(row[:500])
        print()
    print()

****************************** TOPIC 0 ******************************
review_body #1
Buen. Buena calidad, y buena presentación.

review_body #2
Un producto a perfecto, para salir de casa con la Nintendo switch

review_body #3
No funciona con Nintendo Switch. No hay forma de emparejarlo para poder jugar.

review_body #4
Recomendado, los utilizo para pc y no me dan ningún problema, se oye bien y te oyen bien

review_body #5
El cable funciona bien podria ser un poco mas largo pero se adapta perfectamente a la descripcion.envio muy rapido muy recomendado


****************************** TOPIC 1 ******************************
review_body #1
Buen. Buena calidad, y buena presentación.

review_body #2
Un producto a perfecto, para salir de casa con la Nintendo switch

review_body #3
No funciona con Nintendo Switch. No hay forma de emparejarlo para poder jugar.

review_body #4
Recomendado, los utilizo para pc y no me dan ningún problema, se oye bien y te oyen bien

review_body #5
El cable funcio